In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model
#from tensorflow.keras import np_utils 

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import load_model 

import numpy as np
import pickle
import time
import math

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
trainDir = r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train"
#valDir = r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train"

batch_size = 16
height = 256
width = 256
num_classes = 2
dropout_rate = 0.2
initializer = 'he_normal'

def lr_schedule(epoch):
    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 40: # 120
        lr *= 1e-2
    elif epoch > 20: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

def generate_generator_multiple(generator, dir1, dir2, dir3, batch_size, img_height, img_width, subset):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height, img_width),
                                          color_mode='rgb',
                                          batch_size=batch_size,
                                          #class_mode='binary',
                                          shuffle=False,
                                          seed=42,
                                          subset=subset
                                         )
                                          

    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height // 2, img_width // 2),
                                          color_mode='rgb',
                                          batch_size=batch_size,
                                          #class_mode='binary',
                                          shuffle=False,
                                          seed=42,
                                          subset=subset
                                         )
    
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height // 4, img_width // 4),
                                          color_mode='rgb',
                                          batch_size=batch_size,
                                          #class_mode='binary',
                                          shuffle=False,
                                          seed=42,
                                          subset=subset
                                         )
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        X3i = genX3.next()
        yield [X1i[0], X2i[0], X3i[0]], X3i[1]
    
    
train_generator = generate_generator_multiple(generator=train_datagen,
                                              dir1=trainDir,
                                              dir2=trainDir,
                                              dir3=trainDir,
                                              batch_size=batch_size,
                                              img_height=height,
                                              img_width=width,
                                              subset='training'
                                             )

                    
valid_generator = generate_generator_multiple(generator=train_datagen,
                                              dir1=trainDir,
                                              dir2=trainDir,
                                              dir3=trainDir,
                                              batch_size=batch_size,
                                              img_height=height,
                                              img_width=width,
                                              subset='validation'
                                             )                                              

In [4]:
# first model
vgg1 = tf.keras.applications.VGG16(include_top=False, weights=None, input_shape = (224, 224, 3))
vgg1._name = 'vgg1'

#for i, layer in enumerate(vgg1.layers):
    #layer.name = 'layer_a' + str(i)
    
model1 = Sequential()

model1.add(vgg1)

model1.add(GlobalAveragePooling2D())

model1.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

In [5]:
# second model
vgg2 = tf.keras.applications.VGG16(include_top=False, weights=None, input_shape = (128, 128, 3))
vgg2._name = 'vgg2'
#for i, layer in enumerate(vgg2.layers):
    #layer.name = 'layer_b' + str(i)
    
model2 = Sequential()

model2.add(vgg2)

model2.add(GlobalAveragePooling2D())

model2.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

In [6]:
#third model

vgg3 = tf.keras.applications.VGG16(include_top=False, weights=None, input_shape = (64, 64, 3))
vgg3._name = 'vgg3'

#for i, layer in enumerate(vgg3.layers):
    #layer.name = 'layer_c' + str(i)
        
model3 = Sequential()

model3.add(vgg3)

model3.add(GlobalAveragePooling2D())

model3.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

In [7]:
merged_model = Concatenate()([model1.output, model2.output, model3.output])

x = Dense(512, activation='relu')(merged_model)
x = Dropout(dropout_rate)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(dropout_rate)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(dropout_rate)(x)
output = Dense(num_classes)(x)


In [8]:
model = Model(inputs=[model1.input, model2.input, model3.input], outputs=[output])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
vgg1_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg2_input (InputLayer)         [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
vgg3_input (InputLayer)         [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
vgg1 (Functional)               (None, 7, 7, 512)    14714688    vgg1_input[0][0]                 
_______________________________________________________________________________________

In [9]:
#tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_simple/logs/{}".format(NAME))

#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_sparse_categorical_accuracy', mode='max', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr_schedule(0)),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Learning rate:  0.0001


In [10]:
DATA_SIZE = 19680
TRAINING_SIZE = DATA_SIZE*0.8 
VALIDATION_SIZE = DATA_SIZE*0.2

compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))

In [11]:
STEP_SIZE_TRAIN = compute_steps_per_epoch(TRAINING_SIZE)
STEP_SIZE_VALID = compute_steps_per_epoch(VALIDATION_SIZE)

In [12]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=checkpoint)

Instructions for updating:
Please use Model.fit, which supports generators.
Found 15743 images belonging to 2 classes.
Found 15743 images belonging to 2 classes.
Found 15743 images belonging to 2 classes.
Epoch 1/50


InvalidArgumentError:  Can not squeeze dim[1], expected a dimension of 1, got 2
	 [[node Squeeze (defined at <ipython-input-12-649748e510cf>:6) ]] [Op:__inference_train_function_5078]

Function call stack:
train_function
